<a href="https://colab.research.google.com/github/dinianggraini13/SKRIPSI-FIX-FIX/blob/main/Scrapping%20Threads%20Fix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq google-play-scraper

In [ ]:
import json
import pandas as pd
from tqdm import tqdm
from google_play_scraper import Sort, reviews, app

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
path = "gdrive/MyDrive/"

Mounted at /content/gdrive


In [ ]:
app_packages = [
    'com.instagram.barcelona'
]

In [ ]:
app_infos = []

for application in tqdm(app_packages):
  info = app(application, lang='en')
  del info['comments']
  app_infos.append(info)

100%|██████████| 1/1 [00:00<00:00,  4.79it/s]


In [ ]:
from tqdm import tqdm
from google_play_scraper import reviews, Sort
from datetime import datetime

app_reviews = []

for ap in tqdm(app_packages):
        for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
            rvs, _ = reviews(
                ap,
                lang='en',
                sort=sort_order,
                count= 20000
            )

            for r in rvs:
                # Memastikan bahwa 'at' adalah dalam format string tanggal
                if isinstance(r['at'], str):
                    r['at'] = datetime.strptime(r['at'], "%Y-%m-%dT%H:%M:%S.%fZ")

                # Menambahkan informasi order dan ID aplikasi
                r['sortOrder'] = 'most_relevant'
                r['appId'] = ap

            # Memfilter ulasan berdasarkan tanggal (Juli hingga Agustus)
            july_august_reviews = [
                review for review in rvs
                if isinstance(review['at'], datetime) and datetime(2023, 7, 1) <= review['at'] <= datetime(2023, 8, 31)
            ]

            app_reviews.extend(july_august_reviews)



100%|██████████| 1/1 [01:03<00:00, 63.23s/it]


In [ ]:
app_reviews_df = pd.DataFrame(app_reviews)

In [ ]:
app_reviews_df = app_reviews_df.sort_values(['appId','at'])

In [ ]:
app_reviews_df.to_csv(path+'hasilreviews.csv', index=None, header=True)

In [ ]:
len(app_reviews)

34885

In [ ]:
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,sortOrder,appId
6119,068ccedd-f9b4-4b13-acd5-36320e14d29c,Jeffrey Otto,https://play-lh.googleusercontent.com/a-/ALV-U...,"At this time it doesn't allow new accounts, on...",1,71,289.0.0.77.109,2023-07-05 22:53:12,None,None,289.0.0.77.109,most_relevant,com.instagram.barcelona
3188,ed18e46f-0d10-47a7-975c-5daef461e841,Okhale Oziegbe,https://play-lh.googleusercontent.com/a/ACg8oc...,"Competition breeds innovation, despite my dist...",4,41,289.0.0.77.109,2023-07-05 22:59:40,None,None,289.0.0.77.109,most_relevant,com.instagram.barcelona
6923,6a6cd22c-72a0-4a4a-907f-69d4a2a1fd25,Ted Gilroy,https://play-lh.googleusercontent.com/a/ACg8oc...,"Gosh darn, I loved in the movie when they thre...",5,8,289.0.0.77.109,2023-07-05 23:02:05,None,None,289.0.0.77.109,most_relevant,com.instagram.barcelona
7533,96ce81c3-c959-4f12-902f-637dcde01639,Archit Shukla,https://play-lh.googleusercontent.com/a-/ALV-U...,"So far so good ,hope it'll compete well with t...",4,1,289.0.0.77.109,2023-07-05 23:07:03,None,None,289.0.0.77.109,most_relevant,com.instagram.barcelona
3783,c8cbd65d-712d-46c4-939e-b84712c6b146,Martin (iMarbot),https://play-lh.googleusercontent.com/a-/ALV-U...,Home feed full of verified people you don't fo...,2,114,289.0.0.77.109,2023-07-05 23:07:31,None,None,289.0.0.77.109,most_relevant,com.instagram.barcelona


In [ ]:
# Membuat dataframe baru hanya dengan kolom 'reviewId','content' dan 'score'
df_trimmed_reviews = pd.DataFrame({'reviewId': [review['reviewId'] for review in app_reviews],
                                   'content': [review['content'] for review in app_reviews],
                                   'score': [review['score'] for review in app_reviews]})

# Output dataframe baru
print(df_trimmed_reviews)

                                   reviewId  \
0      764601e7-e20c-411e-9224-f9c90e659e15   
1      2e4384be-6c76-4551-b2a5-42bb5fa04419   
2      4efedf5c-dcd5-4c63-b968-74898761efaa   
3      8dd441f6-1c28-4432-bb36-1d569ba7eb75   
4      f312536e-4afa-4e6f-b52d-cadf19a96a1f   
...                                     ...   
34880  45500b7f-d543-4d19-a042-62c9d993a8f8   
34881  279ea7fa-f038-4cb2-9db3-cac9d0ca52f4   
34882  7e5d9929-0442-4522-ad31-9b5084ace949   
34883  f5464f0f-c9d3-410b-afc6-fd654b93a757   
34884  d28c94f2-c471-4dd0-9408-d4fab0dd99d1   

                                                 content  score  
0      Meh. Not the greatest experience on a Chromebo...      2  
1      For a brand new app, it's very well optimized....      3  
2      Great app with a lot of potential! However, th...      3  
3      I literally can't use the app. It won't let me...      1  
4      The platform has potential, but the app, which...      3  
...                                    

In [ ]:
df_trimmed_reviews.to_csv(path+'hasilreviewsFIX.csv', index=None, header=True)